In [1]:
import pandas as pd
import os

read in files

In [2]:
taxa_df = pd.read_csv('taxa.csv')
otu_df = pd.read_csv('otu.csv')
sample_data_df = pd.read_csv('sample_data.csv')

otu_df

,Unnamed: 0,RRN-SKG2-p2-92,RRN-SKG2-p2-95,RRN-SKG2-p2-96
0,768377abe500ad4d09c43a4532e533b1,2071,237,1189
1,c12b77b8b00eee715945f54d3ce0bd87,0,0,0
2,caa314c8bbe94ec49c4b5e7ac761c89c,150,327,435
3,db35c906865fb1665ab2ed219d1e567e,3,0,0
4,c40d1ea1bab89e3c59c66d9ebba95dbd,268,900,427
...,...,...,...,...
1374,0c6be2f1306b70e76f6aba69b1ab5b93,0,0,0
1375,7bbb77961d1e324e1d52013da3040d99,0,0,0
1376,81c8fb6ade4a2a205ebc9edd13c36287,0,0,0
1377,168647e9e55ed8c1153feffe02ab9e26,0,0,0


remove OTUs that are less than 0.1% for all samples

In [3]:
otu_df.rename(columns={ otu_df.columns[0]: 'id' }, inplace = True)
otu_delete_lst = []

otu_lst = otu_df.iloc[:,0].tolist()
for otu in otu_lst:
    otu_row = otu_df.loc[otu_df['id'] == otu]
    otu_values_lst = otu_row.iloc[0, 1:].tolist()
    for value in otu_values_lst:
        if value >= 0.1:
            break
    else:
        otu_delete_lst.append(otu)
        
otu_df = otu_df[~otu_df['id'].isin(otu_delete_lst)]
otu_df

,id,RRN-SKG2-p2-92,RRN-SKG2-p2-95,RRN-SKG2-p2-96
0,768377abe500ad4d09c43a4532e533b1,2071,237,1189
2,caa314c8bbe94ec49c4b5e7ac761c89c,150,327,435
3,db35c906865fb1665ab2ed219d1e567e,3,0,0
4,c40d1ea1bab89e3c59c66d9ebba95dbd,268,900,427
5,cece12865e81c96a523b3bb6a4511561,1109,192,264
...,...,...,...,...
1301,8e9653889841f4ef72e9e3cd159d126e,0,13,0
1302,af2a98dbd879dd3178779443a630917a,0,13,0
1344,115bf860d51e45a9104c03930fb4d6c9,0,12,0
1361,d88962df86cb483b88e7227efb652326,0,0,3


reformat otu column names

In [4]:
taxa_df.rename(columns={ taxa_df.columns[0]: 'id' }, inplace = True)
taxa_df = taxa_df.fillna('unclassified')
otu_lst = otu_df.iloc[:,0].tolist()

for otu in otu_lst:
    otu_row = taxa_df.loc[taxa_df['id'] == otu]
    
    kingdom = otu_row.iloc[0, 1]
    phylum = otu_row.iloc[0, 2]
    clas = otu_row.iloc[0, 3]
    order = otu_row.iloc[0, 4]
    family = otu_row.iloc[0, 5]
    genus = otu_row.iloc[0, 6]
    species = otu_row.iloc[0, 7]
    
    formatted_name = f'k__{kingdom}|p__{phylum}|c__{clas}|o__{order}|f__{family}|g__{genus}|s__{genus}_{species}'
    otu_df.loc[otu_df['id']==otu, 'id'] = formatted_name


otu_df

,id,RRN-SKG2-p2-92,RRN-SKG2-p2-95,RRN-SKG2-p2-96
0,k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__...,2071,237,1189
2,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,150,327,435
3,k__Bacteria|p__Firmicutes|c__Bacilli|o__Lactob...,3,0,0
4,k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__...,268,900,427
5,k__Bacteria|p__Bacteroidota|c__Bacteroidia|o__...,1109,192,264
...,...,...,...,...
1301,k__Bacteria|p__Firmicutes|c__Clostridia|o__Lac...,0,13,0
1302,k__Bacteria|p__Firmicutes|c__Clostridia|o__Osc...,0,13,0
1344,k__Bacteria|p__Firmicutes|c__Clostridia|o__Osc...,0,12,0
1361,k__Bacteria|p__Firmicutes|c__Clostridia|o__Osc...,0,0,3


get labels

In [5]:
targets = []
samples = otu_df.columns[1:]
for sample in samples:
    target = sample_data_df.loc[sample_data_df['SampleLabel'] == sample, 'exp'].iloc[0]
    target2 = sample_data_df.loc[sample_data_df['SampleLabel'] == sample, 'TimePoint'].iloc[0]

    
    
    #SKGNak -> 0
    #WT -> 1
    #SKGBoyJ ->2
    #if target == 1 & (target2 == 3):
    if target == 'SKG2':
        targets.append(1)
    else:
        targets.append(0)
    # elif 'SKGBoyJ' in target:
    #label = 2
    #targets.append(label)

    
targets

[0, 0, 0]

export dataframes to tsv and txt files

In [6]:
folder_name = 'SKG2-wt-t14' #EDIT

otu_df.to_csv('../data/' + folder_name + '/abundance.tsv', sep="\t", header=None,index=False)
with open('../data/' + folder_name + '/labels.txt', 'w') as f:
    for line in targets:
        f.write(f"{line}\n")